In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from scipy.io import loadmat

In [11]:
import os
files = sorted(os.listdir('../_data/genomics-data/DataCV/Raw'))

In [14]:
files[0], files[1]

('CTNNB1_All(0.01).trn.mat', 'CTNNB1_All(0.01).tst.mat')

In [18]:
train = loadmat(f'../_data/genomics-data/DataCV/Raw/{files[0]}')
test  = loadmat(f'../_data/genomics-data/DataCV/Raw/{files[1]}')

In [20]:
train.keys()

dict_keys(['__header__', '__version__', '__globals__', 'DatasetTrn'])

In [112]:
# 0: Positive class data (16662, 18)
# 1: Negative class data (673200, 18)
# 2: Positive class additional info [Cause Gene, Effect Gene, Replicate, Treatment, Pvalue]
# 3: Negative class additional info [Cause Gene, Effect Gene, Replicate, Treatment, Pvalue]
# 4: Column names (separated by ;)
# 5: [[6]]

for i, v in enumerate(train['DatasetTrn'][0,0]):
    if i in [2,3]:
        for w in v[0]:
            print(w.shape)
    else:
        print(v.shape)

(16662, 18)
(673200, 18)
(16662, 1)
(16662, 1)
(16662, 1)
(16662, 1)
(16662, 1)
(673200, 1)
(673200, 1)
(673200, 1)
(673200, 1)
(673200, 1)
(1,)
(1, 1)


In [147]:
train['DatasetTrn'][0,0][5]

array([[6]], dtype=uint8)

In [124]:
for i, v in enumerate(test['Dataset'][0,0]):
    if i in [2,3]:
        for w in v[0]:
            print(w.shape)
    else:
        print(v.shape)

(60, 18)
(81876, 18)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(81876, 1)
(81876, 1)
(81876, 1)
(81876, 1)
(81876, 1)
(1,)
(1, 1)


In [126]:
test['Dataset'][0,0][5]

array([[6]], dtype=uint8)

In [158]:
columns = test['Dataset'][0,0][4][0].strip().split(';')

['CauseGene',
 'EffectGene',
 'Replicate',
 'Treatment',
 'Pvalue',
 '0min_neg_cause',
 '15min_cause',
 '30min_cause',
 '90min_cause',
 '120min_cause',
 '180min_cause',
 '210min_cause',
 '240min_cause',
 '360min_cause',
 '0min_neg_effect',
 '15min_effect',
 '30min_effect',
 '90min_effect',
 '120min_effect',
 '180min_effect',
 '210min_effect',
 '240min_effect',
 '360min_effect']

In [154]:
train.keys()

dict_keys(['__header__', '__version__', '__globals__', 'DatasetTrn'])

In [161]:
df = pd.DataFrame(columns=columns)

In [206]:
len(train['DatasetTrn'][0,0][1])

673200

In [209]:
import re

def mat_to_dataframe(mat_object):
    key = [key for key in mat_object if 'Dataset' in key][0]
    columns = mat_object[key][0,0][4][0].strip().split(';')
    df = pd.DataFrame()
    for i in range(5):
        df[columns[i]] = np.concatenate([
                        mat_object[key][0,0][2][0][i].reshape(-1),
                        mat_object[key][0,0][3][0][i].reshape(-1)
        ])
    for i in range(18):
        df[columns[i+5]] = np.concatenate([
                        mat_object[key][0,0][0][:,i],
                        mat_object[key][0,0][1][:,i]
        ])
    for i in [0, 1, 3]:
        df[columns[i]] = df[columns[i]].apply(lambda x: x[0])
    df['Target'] = [0]*len(mat_object[key][0,0][0]) + \
                   [1]*len(mat_object[key][0,0][1])
    return df

In [210]:
train_df = mat_to_dataframe(train)

In [211]:
train_df

,CauseGene,EffectGene,Replicate,Treatment,Pvalue,0min_neg_cause,15min_cause,30min_cause,90min_cause,120min_cause,...,0min_neg_effect,15min_effect,30min_effect,90min_effect,120min_effect,180min_effect,210min_effect,240min_effect,360min_effect,Target
0,MAPK1,AGTRAP,1,BCR,0.008,1.570461,-0.259946,0.461187,-1.821926,0.217530,...,1.343140,0.967046,0.550347,-0.396333,0.762589,-0.857914,-0.271856,-1.741207,0.751194,0
1,MAPK1,AGTRAP,1,CD40,0.008,1.570461,0.540413,-0.058239,-0.384124,0.131689,...,1.343140,0.141576,0.677390,-1.477123,-1.339167,-1.487985,0.477539,-0.191964,-0.440323,0
2,MAPK1,AGTRAP,2,BCR,0.008,0.814767,-1.303725,0.780416,1.014104,0.493707,...,1.179496,1.347345,0.476586,-0.944036,-0.031326,-1.069956,-0.939858,-0.994739,-1.215449,0
3,MAPK1,AGTRAP,2,CD40,0.008,0.814767,-1.453817,0.690254,0.163129,0.340793,...,1.179496,0.093155,0.319289,-0.816700,-1.353020,0.005323,-0.141223,-0.072680,1.467384,0
4,MAPK1,AGTRAP,3,BCR,0.008,-1.276183,1.060551,0.513360,-0.863885,-0.520006,...,0.755433,1.313819,1.005074,-1.053026,-0.192625,-0.763318,-0.270895,-0.908896,0.380477,0
5,MAPK1,AGTRAP,3,CD40,0.008,-1.276183,-3.544150,0.386078,0.293890,0.756262,...,0.755433,1.025937,0.369601,-1.445271,-1.736670,-1.528987,0.523889,-0.440154,1.043925,0
6,MAPK1,CHAC1,1,BCR,0.008,1.570461,-0.259946,0.461187,-1.821926,0.217530,...,-0.258804,-0.103963,0.039941,-0.212841,0.871549,1.667838,2.095155,1.446754,1.831465,0
7,MAPK1,CHAC1,1,CD40,0.008,1.570461,0.540413,-0.058239,-0.384124,0.131689,...,-0.258804,-0.082021,-0.150566,-0.841216,-1.039542,-1.005486,-0.704339,-0.869828,-0.852197,0
8,MAPK1,CHAC1,2,BCR,0.008,0.814767,-1.303725,0.780416,1.014104,0.493707,...,0.693092,-0.268454,-0.137050,0.993101,0.358369,0.910697,0.915863,1.872587,1.036609,0
9,MAPK1,CHAC1,2,CD40,0.008,0.814767,-1.453817,0.690254,0.163129,0.340793,...,0.693092,-0.207244,-0.200419,-0.864206,-0.932040,-0.928704,-0.829367,-1.010154,-0.580542,0


In [212]:
test_df = mat_to_dataframe(test)

In [213]:
test_df

,CauseGene,EffectGene,Replicate,Treatment,Pvalue,0min_neg_cause,15min_cause,30min_cause,90min_cause,120min_cause,...,0min_neg_effect,15min_effect,30min_effect,90min_effect,120min_effect,180min_effect,210min_effect,240min_effect,360min_effect,Target
0,CTNNB1,CSRP2,1,BCR,0.000,-0.105283,-0.541612,-0.393067,-1.174035,-0.107926,...,-0.041915,1.175773,-0.909400,0.497062,0.291227,-0.637650,-0.786138,-0.667207,0.407988,0
1,CTNNB1,CSRP2,1,CD40,0.000,-0.105283,-0.347395,-0.950074,-1.130433,-0.297189,...,-0.041915,0.010091,0.395842,0.919178,-0.347473,-0.746976,0.839197,-0.424967,-0.324002,0
2,CTNNB1,CSRP2,2,BCR,0.000,-0.461648,-1.277333,-0.407660,1.186022,0.871369,...,1.220617,0.406977,0.388063,-0.083557,-1.994005,-0.156349,2.909207,-1.256498,-2.252847,0
3,CTNNB1,CSRP2,2,CD40,0.000,-0.461648,-1.710065,-0.950477,-0.845131,-0.700774,...,1.220617,1.120649,0.254863,0.118594,0.498969,0.517242,-0.097572,0.077523,0.457753,0
4,CTNNB1,CSRP2,3,BCR,0.000,-1.374423,-0.282157,0.046609,0.002050,-0.075820,...,0.591445,-0.375086,-0.543964,1.835650,0.403146,0.010032,-0.540943,-1.429848,-0.674091,0
5,CTNNB1,CSRP2,3,CD40,0.000,-1.374423,-2.523015,-0.684827,-0.598375,0.068134,...,0.591445,3.721650,0.418995,0.125605,-0.148227,-0.115194,-0.374170,0.104104,-0.781838,0
6,CTNNB1,DDIT4,1,BCR,0.002,-0.105283,-0.541612,-0.393067,-1.174035,-0.107926,...,0.152078,-0.047317,-0.064806,-0.353372,-0.055486,0.653737,1.437274,1.637604,2.319942,0
7,CTNNB1,DDIT4,1,CD40,0.002,-0.105283,-0.347395,-0.950074,-1.130433,-0.297189,...,0.152078,-0.236969,-0.568937,-1.339564,-1.231009,-0.910465,-0.745417,-0.668043,-0.299993,0
8,CTNNB1,DDIT4,2,BCR,0.002,-0.461648,-1.277333,-0.407660,1.186022,0.871369,...,0.335872,0.057069,-0.254336,-0.063568,0.024954,0.016735,0.837676,0.933870,2.588862,0
9,CTNNB1,DDIT4,2,CD40,0.002,-0.461648,-1.710065,-0.950477,-0.845131,-0.700774,...,0.335872,0.044189,-0.443012,-1.160505,-1.091359,-0.705328,-0.642899,-0.416881,-0.880110,0


## Test utils

In [214]:
from utils import load_mat
df = load_mat(f'../_data/genomics-data/DataCV/Raw/{files[0]}')

In [216]:
df.shape

(689862, 24)

In [215]:
df.sample(5)

,CauseGene,EffectGene,Replicate,Treatment,Pvalue,0min_neg_cause,15min_cause,30min_cause,90min_cause,120min_cause,...,0min_neg_effect,15min_effect,30min_effect,90min_effect,120min_effect,180min_effect,210min_effect,240min_effect,360min_effect,Target
606000,LEF1,OSR2,1,BCR,0.997,0.777138,0.621560,0.762682,0.949396,-0.388986,...,0.311859,0.372335,0.180680,0.188024,-1.164472,-1.508297,-0.995302,-1.220121,-1.165186,1
222918,NFKB2,EMC3,1,BCR,0.889,-0.840552,-0.995702,-0.447665,0.566654,0.124721,...,-0.023274,-0.271412,-0.317501,-1.881324,0.249419,0.341617,1.025382,-0.130291,0.733856,1
135559,MAPK3,PSMD4,1,CD40,0.861,0.246210,0.602832,-0.412384,-0.463134,-1.677173,...,1.647301,0.332301,-0.024842,-0.962431,-0.401319,0.079930,-0.118112,-0.387605,-0.937120,1
610300,LEF1,PRDX3,3,BCR,0.988,0.047143,0.630933,0.890430,-0.268958,-0.187994,...,-1.057760,0.699630,0.251152,-1.443747,0.040831,-1.183490,-0.432082,1.940664,-1.383130,1
157943,NFKB1,ADIPOR1,3,CD40,0.542,-1.647137,-2.017196,-1.074282,-0.109298,0.491217,...,-1.006475,-1.737100,0.571520,0.480307,0.081334,0.362462,0.088105,0.275897,0.795297,1
